<a href="https://colab.research.google.com/github/gsiolas/nn-notebooks/blob/main/05_Scipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightspeed introduction to `scipy`

![](https://www.fullstackpython.com/img/logos/scipy.png)

The `scipy` library is one of the core packages for scientific computing with Python.

It provides many user-friendly and efficient numerical routines such as routines for numerical integration and optimization.

Library documentation: <a>http://www.scipy.org/</a>

## Numerical integration

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from scipy.integrate import quad

x = np.linspace(-10, 10, 1000)
y = np.exp(- x ** 2 / 2)
_ = plt.plot(x, y)

In [ ]:
# Integration
val, abserr = quad(lambda x: np.exp(-x ** 2 / 2),  -np.inf, np.inf)
val, (2 * np.pi) ** 0.5, abserr, 

## Optimization

In [ ]:
def f(x):
    return 4*x**3 + (x-2)**2 + x**4

x = np.linspace(-5, 3, 100)
_ = plt.plot(x, f(x))

In [ ]:
import scipy as sp

x_min = sp.optimize.fmin_bfgs(f, -2)
print(x_min)

In [ ]:
x_min = sp.optimize.fmin_bfgs(f, 3)
print(x_min)

## Statistics

In [ ]:
from scipy import stats

Y = stats.norm()
x = np.linspace(-5, 5, 100)
plt.figure(figsize=(4, 4))
plt.subplot(3, 1, 1)
plt.plot(x, Y.pdf(x)) # PDF function
plt.subplot(3, 1, 2)
plt.plot(x, Y.cdf(x)) # CDF function
plt.subplot(3, 1, 3)
_ = plt.hist(Y.rvs(size=1000), bins=40) # histogram of simulations
plt.tight_layout()

In [ ]:
Y.mean(), Y.std(), Y.var()

In [ ]:
# t-test example
t_statistic, p_value = stats.ttest_ind(Y.rvs(size=1000), Y.rvs(size=1000))
t_statistic, p_value

## Exercice. MLE of a Gamma distribution

Let us recall that the density of a $\Gamma(a, \lambda)$ (Gamma distribution with shape parameter $a > 0$ and intensity $\lambda > 0$ is given by

$$
f_{a, \lambda}(x) = \frac{\lambda^a}{\Gamma(a)} x^{a - 1} e^{-\lambda x} \mathbf 1_{x \geq 0}
$$

1. Look for this distribution in `scipy` and plot its density function with shape=4.2 and intensity=1.5
1. Simulate a i.i.d sample of size $n=2000$ with this distribution
1. Write a function which computes the negative log-likelihood of this samples with prototype `neg_log_lik(params)` where `params` is an array of shape `(2,)` containing the scale and intensity
1. Use `scipy.optim` to minimize this function. Check that your estimator seems correct. Try to be clever with the initialization

In [ ]:
from scipy.stats import gamma

# Shape and intensity parameters
a, intensity = 4.2, 1.5

# Scale is 1 / intensity
G = gamma(a=a, scale=1 / intensity)

In [ ]:
# Let's check that scale is indeed 1 / intensity
G.mean(), a / intensity

In [ ]:
x = np.linspace(1e-4, 10, 1000)
plt.figure(figsize=(6, 6))
plt.subplot(3, 1, 1)
_ = plt.plot(x, G.pdf(x))
plt.subplot(3, 1, 2)
_ = plt.plot(x, G.cdf(x))
plt.subplot(3, 1, 3)
_ = plt.hist(G.rvs(size=2000), bins=25)
plt.tight_layout()

In [ ]:
# Let's sample 2000 realizations of this Gamma distribution
X = G.rvs(size=2000)

In [ ]:
def neg_log_lik(params, X):
    """Negative log-likelihood of a Gamma distribution
    
    Parameters
    ---------
    params : `np.ndarray`, shape=(2,)
        params[0] is parameter a of the Gamma distribution
        params[1] is the intensity (1/scale) of the Gamma 
        distribution

    X : `np.ndarray`, shape=(n_samples,)
        The samples on which the negative log-likelihood is
        computed
    
    Returns
    -------
    output : `float`
        The value of the negative log-likelihood
    """
    a, intensity = params
    return -np.log(gamma.pdf(X, a=a, scale=1 / intensity)).mean()

In [ ]:
neg_log_lik(np.array([4.2, 1.5]), X)

In [ ]:
neg_log_lik(np.array([6., 1.]), X)

We use the fact that if $X \sim \Gamma(a, \lambda)$ then $\mathbb E X = \frac{a}{\lambda}$ and $\text{var}(X) = \frac{a}{\lambda^2}$ so that
$\lambda = \frac{\mathbb E X}{\text{var}(X)}$ and $a = \mathbb E X \times \lambda$, so that we can use the methods of moments.


In [ ]:
X.mean() / X.var()

In [ ]:
X.mean() * intensity

In [ ]:
def mle_gamma(X, verbose=False):
    """Compute the maximum likelihood estimator (MLE) of the Gamma
    distribution, using an initialization based on the method of
    moments
    
    Parameters
    ----------
    X : `np.ndarray`, shape=(n_samples,)
        The samples on which we apply MLE for the Gamma 
        distribution
    
    verbose : `bool`
        If `True`, display information about convergence,
        otherwise STFU
        
    Returns
    -------
    output : `tuple[float, float]`
        The estimated (a, intensity) parameter        
    """
    from scipy.optimize import fmin_bfgs

    # We initialize the solver using the method of moments
    intensity0 = X.mean() / X.var()
    a0 = X.mean() * intensity0

    a, intensity = fmin_bfgs(
        lambda params: neg_log_lik(params, X),
        np.array([a0, 1 / intensity0]),
        full_output=0,
        disp=verbose
    )
    return a, intensity

In [ ]:
mle_gamma(X, verbose=True)

In [ ]:
gamma.fit(X, floc=0)